# Fourier Power Spectra Analysis

Interactive notebook for analyzing raw voltage recordings using FFT power spectra.
This notebook is designed to work with electrophysiology data recorded as `.dat` files.

## 1. Setup and Imports

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import sys

# Add the current directory to Python path to import our custom module
sys.path.append('.')

# Import our Fourier analysis module
from fourier_analysis import VoltageRecordingAnalyzer, get_default_frequency_bands, quick_analysis

print("Setup complete!")
print("Available functions:")
print("- VoltageRecordingAnalyzer: Main analysis class")
print("- quick_analysis: Fast analysis function")
print("- get_default_frequency_bands: Get standard frequency bands")

Setup complete!
Available functions:
- VoltageRecordingAnalyzer: Main analysis class
- quick_analysis: Fast analysis function
- get_default_frequency_bands: Get standard frequency bands


## 2. File Path Configuration

In [ ]:
# Configure file path
# Default path - you can modify this or use the widget below
default_path = "C:\\Users\\wanglab\\Desktop\\Club Like Endings\\040425_1\\amplifier.dat"

# Create file path widget
file_path_widget = widgets.Text(
    value=default_path,
    placeholder='Enter path to .dat file',
    description='File Path:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Sampling rate widget
sampling_rate_widget = widgets.IntText(
    value=30000,
    description='Sampling Rate (Hz):',
    style={'description_width': 'initial'}
)

# Number of channels widget
n_channels_widget = widgets.IntText(
    value=64,
    description='Number of Channels:',
    style={'description_width': 'initial'}
)

display(widgets.VBox([
    widgets.HTML("<h3>Recording Configuration</h3>"),
    file_path_widget,
    widgets.HBox([sampling_rate_widget, n_channels_widget])
]))

print("File path configuration ready!")
print(f"Default path will be converted to WSL format: /mnt/c/Users/wanglab/Desktop/Club Like Endings/040425_1/amplifier.dat")

## 3. Quick Analysis

In [ ]:
# Quick analysis controls
duration_widget = widgets.FloatSlider(
    value=10.0,
    min=1.0,
    max=60.0,
    step=1.0,
    description='Duration (s):',
    style={'description_width': 'initial'}
)

channel_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=63,
    description='Channel:',
    style={'description_width': 'initial'}
)

start_time_widget = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=300.0,
    step=1.0,
    description='Start Time (s):',
    style={'description_width': 'initial'}
)

quick_analysis_button = widgets.Button(
    description='Run Quick Analysis',
    button_style='success',
    layout=widgets.Layout(width='200px')
)

def run_quick_analysis(button):
    with quick_output:
        clear_output(wait=True)
        
        try:
            print("Starting quick analysis...")
            file_path = file_path_widget.value
            
            # Create analyzer
            analyzer = VoltageRecordingAnalyzer(sampling_rate=sampling_rate_widget.value)
            
            # Load data
            print("Loading data...")
            data, time = analyzer.load_amplifier_data(
                file_path,
                n_channels=n_channels_widget.value,
                duration_sec=duration_widget.value,
                start_sec=start_time_widget.value
            )
            
            # Analyze channel
            print(f"Analyzing channel {channel_widget.value}...")
            freq_bands = get_default_frequency_bands()
            results = analyzer.analyze_channel(channel_widget.value, freq_bands=freq_bands, plot=True)
            
            # Print summary
            print("\n" + "="*50)
            print("ANALYSIS SUMMARY")
            print("="*50)
            print(f"Channel: {results['channel']}")
            print(f"Peak frequency: {results['peak_frequency']:.2f} Hz")
            print(f"Total power: {results['total_power']:.2e}")
            
            if results['band_powers']:
                print("\nFrequency band powers:")
                for band, power in results['band_powers'].items():
                    print(f"  {band:>12}: {power:.2e}")
            
            # Store results globally for further analysis
            global last_analyzer, last_results
            last_analyzer = analyzer
            last_results = results
            
            print("\nAnalysis complete! Results stored for further analysis.")
            
        except Exception as e:
            print(f"Error during analysis: {str(e)}")
            print("\nTroubleshooting tips:")
            print("1. Check that the file path is correct")
            print("2. Ensure the file exists and is accessible")
            print("3. Verify the number of channels is correct")
            print("4. Make sure you have enough memory for the analysis")

quick_analysis_button.on_click(run_quick_analysis)

# Display controls
display(widgets.VBox([
    widgets.HTML("<h3>Quick Analysis Parameters</h3>"),
    widgets.HBox([duration_widget, channel_widget]),
    start_time_widget,
    quick_analysis_button
]))

quick_output = widgets.Output()
display(quick_output)

## 4. Multi-Channel Analysis

In [ ]:
# Multi-channel analysis controls
max_channels_widget = widgets.IntSlider(
    value=16,
    min=1,
    max=64,
    description='Max Channels:',
    style={'description_width': 'initial'}
)

multi_duration_widget = widgets.FloatSlider(
    value=5.0,
    min=1.0,
    max=30.0,
    step=1.0,
    description='Duration (s):',
    style={'description_width': 'initial'}
)

multi_start_widget = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=300.0,
    step=1.0,
    description='Start Time (s):',
    style={'description_width': 'initial'}
)

multi_analysis_button = widgets.Button(
    description='Run Multi-Channel Analysis',
    button_style='primary',
    layout=widgets.Layout(width='250px')
)

def run_multi_analysis(button):
    with multi_output:
        clear_output(wait=True)
        
        try:
            print("Starting multi-channel analysis...")
            file_path = file_path_widget.value
            
            # Create analyzer
            analyzer = VoltageRecordingAnalyzer(sampling_rate=sampling_rate_widget.value)
            
            # Load data
            print("Loading data...")
            data, time = analyzer.load_amplifier_data(
                file_path,
                n_channels=n_channels_widget.value,
                duration_sec=multi_duration_widget.value,
                start_sec=multi_start_widget.value
            )
            
            # Analyze all channels
            freq_bands = get_default_frequency_bands()
            all_results = analyzer.analyze_all_channels(
                max_channels=max_channels_widget.value, 
                freq_bands=freq_bands
            )
            
            # Store results globally
            global last_multi_analyzer, last_multi_results
            last_multi_analyzer = analyzer
            last_multi_results = all_results
            
            print(f"\nAnalysis complete! Analyzed {len(all_results)} channels.")
            print("Results stored for further analysis.")
            
        except Exception as e:
            print(f"Error during multi-channel analysis: {str(e)}")
            print("\nTry reducing the number of channels or duration if you run out of memory.")

multi_analysis_button.on_click(run_multi_analysis)

# Display controls
display(widgets.VBox([
    widgets.HTML("<h3>Multi-Channel Analysis Parameters</h3>"),
    widgets.HTML("<p><i>Note: This analyzes multiple channels and may take longer</i></p>"),
    widgets.HBox([max_channels_widget, multi_duration_widget]),
    multi_start_widget,
    multi_analysis_button
]))

multi_output = widgets.Output()
display(multi_output)

## 5. Custom Analysis

In [ ]:
# Custom analysis - run your own code with the loaded data
print("Custom Analysis Section")
print("=" * 30)
print("Use this cell to run custom analysis with the loaded data.")
print("Available variables after running analysis above:")
print("- last_analyzer: VoltageRecordingAnalyzer instance")
print("- last_results: Results from single channel analysis")
print("- last_multi_analyzer: Multi-channel analyzer instance")
print("- last_multi_results: Results from multi-channel analysis")
print("\nExample usage:")
print("analyzer = last_analyzer")
print("data = analyzer.data  # Raw voltage data (samples x channels)")
print("time = analyzer.time  # Time vector")
print("sampling_rate = analyzer.sampling_rate")

# Initialize variables to avoid errors
last_analyzer = None
last_results = None
last_multi_analyzer = None
last_multi_results = None

In [ ]:
# Example custom analysis - modify as needed
if last_analyzer is not None:
    print("Running custom analysis example...")
    
    # Get data
    data = last_analyzer.data
    time = last_analyzer.time
    
    # Example: Compare power spectra of different channels
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Custom Analysis: Channel Comparison', fontsize=16)
    
    channels_to_compare = [0, 1, 2, 3]  # Modify as needed
    colors = ['blue', 'red', 'green', 'orange']
    
    for i, ch in enumerate(channels_to_compare[:4]):
        if ch < data.shape[1]:  # Check if channel exists
            # Compute power spectrum
            freqs, psd = last_analyzer.compute_power_spectrum(data[:, ch])
            
            # Plot in subplots
            ax = axes[i//2, i%2]
            ax.loglog(freqs[1:], psd[1:], color=colors[i], label=f'Channel {ch}')
            ax.set_xlabel('Frequency (Hz)')
            ax.set_ylabel('PSD')
            ax.set_title(f'Channel {ch} Power Spectrum')
            ax.grid(True, alpha=0.3)
            ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Example: Compute cross-channel correlations
    if data.shape[1] > 1:
        print("\nComputing cross-channel correlations...")
        n_compare = min(4, data.shape[1])
        correlation_matrix = np.corrcoef(data[:, :n_compare].T)
        
        plt.figure(figsize=(8, 6))
        plt.imshow(correlation_matrix, cmap='coolwarm', vmin=-1, vmax=1)
        plt.colorbar(label='Correlation')
        plt.title(f'Cross-Channel Correlation Matrix (Channels 0-{n_compare-1})')
        plt.xlabel('Channel')
        plt.ylabel('Channel')
        plt.show()
        
        print(f"Correlation matrix shape: {correlation_matrix.shape}")
        print(f"Max correlation (off-diagonal): {np.max(correlation_matrix[~np.eye(correlation_matrix.shape[0], dtype=bool)]):.3f}")
        
else:
    print("No data loaded yet. Run the quick analysis or multi-channel analysis first.")

## 6. Export Results

In [ ]:
# Export results to files
import pickle
import json
from datetime import datetime

def export_results_to_file():
    """Export analysis results to files."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Export single channel results
    if 'last_results' in globals() and last_results is not None:
        # Save as pickle (preserves numpy arrays)
        pickle_filename = f"fourier_single_results_{timestamp}.pkl"
        with open(pickle_filename, 'wb') as f:
            pickle.dump(last_results, f)
        print(f"Single channel results saved to: {pickle_filename}")
        
        # Save summary as JSON
        json_summary = {
            'channel': int(last_results['channel']),
            'peak_frequency_hz': float(last_results['peak_frequency']),
            'total_power': float(last_results['total_power']),
            'band_powers': {k: float(v) for k, v in last_results['band_powers'].items()},
            'analysis_timestamp': timestamp
        }
        
        json_filename = f"fourier_single_summary_{timestamp}.json"
        with open(json_filename, 'w') as f:
            json.dump(json_summary, f, indent=2)
        print(f"Single channel summary saved to: {json_filename}")
    
    # Export multi-channel results
    if 'last_multi_results' in globals() and last_multi_results is not None:
        # Save as pickle
        multi_pickle_filename = f"fourier_multi_results_{timestamp}.pkl"
        with open(multi_pickle_filename, 'wb') as f:
            pickle.dump(last_multi_results, f)
        print(f"Multi-channel results saved to: {multi_pickle_filename}")
        
        # Save summary as JSON
        multi_summary = {
            'n_channels_analyzed': len(last_multi_results),
            'channels': {},
            'analysis_timestamp': timestamp
        }
        
        for ch, results in last_multi_results.items():
            multi_summary['channels'][str(ch)] = {
                'peak_frequency_hz': float(results['peak_frequency']),
                'total_power': float(results['total_power']),
                'band_powers': {k: float(v) for k, v in results['band_powers'].items()}
            }
        
        multi_json_filename = f"fourier_multi_summary_{timestamp}.json"
        with open(multi_json_filename, 'w') as f:
            json.dump(multi_summary, f, indent=2)
        print(f"Multi-channel summary saved to: {multi_json_filename}")

export_button = widgets.Button(
    description='Export Results',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

def on_export_click(button):
    with export_output:
        clear_output(wait=True)
        try:
            export_results_to_file()
        except Exception as e:
            print(f"Error during export: {str(e)}")

export_button.on_click(on_export_click)

display(widgets.VBox([
    widgets.HTML("<h3>Export Results</h3>"),
    widgets.HTML("<p>Export analysis results to files (.pkl for full data, .json for summaries)</p>"),
    export_button
]))

export_output = widgets.Output()
display(export_output)

## 7. Analysis Notes

### Tips for Effective Analysis:

1. **Start Small**: Begin with short durations (5-10 seconds) to test your setup
2. **Check File Path**: Make sure the Windows path is correctly converted to WSL format
3. **Memory Management**: Large files may require analyzing shorter segments or fewer channels
4. **Sampling Rate**: Verify the correct sampling rate for your recording system
5. **Channel Configuration**: Confirm the number of channels in your recording

### Frequency Bands:
- **Delta**: 0.5-4 Hz (slow oscillations)
- **Theta**: 4-8 Hz (theta rhythm)
- **Alpha**: 8-13 Hz (alpha rhythm)
- **Beta**: 13-30 Hz (beta activity)
- **Gamma**: 30-100 Hz (gamma oscillations)
- **High Gamma**: 100-300 Hz (high-frequency activity)
- **Ripples**: 150-250 Hz (sharp-wave ripples)

### Troubleshooting:
- **File not found**: Check WSL path conversion and file permissions
- **Memory errors**: Reduce duration or number of channels
- **Slow performance**: Use shorter time segments for analysis
- **No plots appearing**: Try running the cell again or restart the kernel